### Iter protocol

#### For loop under the hood

In [ ]:
%%HTML
<iframe src='https://gfycat.com/ifr/YearlyWelcomeBlowfish' frameborder='0' scrolling='no' allowfullscreen width='640' height='1185'></iframe>

In [ ]:
for value in [1, 2, 3]:
    print(value)

In [ ]:
it = iter([1, 2, 3])
try:
    while True:
        value = next(it)
        print(value)
except StopIteration:
    pass

How to create iterable object? Implement `__iter__` that returns object implementing `__next__`

In [ ]:
class Counter:
    def __init__(self, low, high):
        self.current = low
        self.high = high

    def __iter__(self):
        return self

    def __next__(self):
        if self.current > self.high:
            raise StopIteration
        else:
            self.current += 1
            return self.current - 1


for i in Counter(3, 8):
    print(i)

You don't have to return self in `__iter__`

In [ ]:
class Counter:
    def __iter__(self):
        return iter([1, 2, 3])

for value in Counter():
    print(value)

### Generators

Generator is just like a container, but values are generated on the fly as you iterate

In [ ]:
generator = range(10000000)
big_list = list(generator)

from sys import getsizeof

print(getsizeof(generator))
print(getsizeof(big_list))

Generator comprehensions:

In [ ]:
%%timeit
power_2 = [i**2 for i in range(10**6)]

In [ ]:
%%timeit
power_2_gen = (i**2 for i in range(10**6))

### Yield

In [ ]:
def some_generator():
    print("Starting")
    yield 1
    print("Let's come back to where we left off")
    yield 2
    print("Nope. No more yields")

In [ ]:
gen = some_generator()

In [ ]:
next(gen)

In [ ]:
next(gen)

In [ ]:
next(gen)

In [ ]:
list(some_generator())

In [ ]:
def primitive_range(start: int, stop: int, step: int = 1):
    current = start
    while current < stop:
        yield current
        current += step


for i in primitive_range(0, 4):
    print(i)

In [ ]:
def infinite_power_2_gen():
    current = 2
    while True:
        yield current
        current *= 2

In [ ]:
powers_of_2 = infinite_power_2_gen()
first_4 = powers_of_2[:4]

In [ ]:
from itertools import islice
first_4_powers_gen = islice(infinite_power_2_gen(), 4)
list(first_4_powers_gen)

#### Data pipelines

In [ ]:
!cat techcrunch.csv

In [ ]:
%%HTML
<blockquote class="reddit-card" data-card-created="1570178477"><a href="https://www.reddit.com/r/ProgrammerHumor/comments/ct9942/cries_in_segmentation_fault/">Cries in segmentation fault..</a> from <a href="http://www.reddit.com/r/ProgrammerHumor">r/ProgrammerHumor</a></blockquote>
<script async src="//embed.redditmedia.com/widgets/platform.js" charset="UTF-8"></script>

In [ ]:
file_name = "techcrunch.csv"
lines = (line for line in open(file_name))
list_line = (s.rstrip().split(",") for s in lines)
cols = next(list_line)
company_dicts = (dict(zip(cols, data)) for data in list_line)
funding = (
    int(company_dict["raisedAmt"])
    for company_dict in company_dicts
    if company_dict["round"] == "A"
)
total_series_a = sum(funding)
print(f"Total series A fundraising: ${total_series_a}")


### "My function is not called" :(

In [ ]:
def useful_function():
    print("""Nie ma czegoś takiego jak publiczne pieniądze. 
          Jeśli rząd mówi, że komuś coś da, to znaczy, że zabierze tobie, 
          bo rząd nie ma żadnych własnych pieniędzy""")
    yield 1
    # lots of other code
    return 10
    
x = useful_function()

In [ ]:
def counter():
    current = 0
    while True:
        next_val = yield current
        if next_val is not None:
            current = next_val
        current += 1


c = counter()

In [ ]:
next(c)

In [ ]:
next(c)

In [ ]:
c.send(-100)

In [ ]:
next(c)

In [ ]:
c.throw(RuntimeError("Sorry"))

In [ ]:
c.close()

### Context managers - RAII in Python

In [ ]:
with open("irrelevant.txt","w") as file:
    file.write("raii")

Is better than:

In [ ]:
file = open("irrelevant.txt","w")
try:
    file.write("raii")
finally:
    file.close()

In [ ]:
from threading import Lock

lock = Lock()
x = 10

In [ ]:
lock.acquire()
x += 1
lock.release()

In [ ]:
with lock:
    x += 1
with lock:
    x += 1

In [ ]:
%%HTML
<blockquote class="reddit-card" data-card-created="1570178622"><a href="https://www.reddit.com/r/ProgrammerHumor/comments/bfr1xc/i_love_python_but/">I love Python, but...</a> from <a href="http://www.reddit.com/r/ProgrammerHumor">r/ProgrammerHumor</a></blockquote>
<script async src="//embed.redditmedia.com/widgets/platform.js" charset="UTF-8"></script>

#### How does this work under the hood?

In [ ]:
class File:
    def __init__(self, name: str, mode: str = "r"):
        self.name = name
        self.mode = mode
        self.file_handle = None

    def __enter__(self):
        self.file_handle = open(self.name, self.mode)
        return self.file_handle

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("__exit__ called")
        if self.file_handle:
            self.file_handle.close()


with File("irrelevant.txt", "r") as f:
    10 / 0

### contextlib

In [ ]:
from contextlib import contextmanager


@contextmanager
def File(name: str, mode: str = "r"):
    file_handle = None
    try:
        file_handle = open(name, mode)
        yield file_handle
    finally:
        if file_handle:
            file_handle.close()

In [ ]:
with File("irrelevant.txt", "r") as f:
    10 / 0

In [ ]:
with File("3.txt", "r") as f:
    10 / 0

In [ ]:
import sys
import datetime
from typing import Generator
from typing.io import TextIO
from contextlib import contextmanager



@contextmanager
def execution_time_printed(file: TextIO = sys.stdout) -> Generator[None, None, None]:
    start = datetime.datetime.now()
    yield
    print("Execution time:", datetime.datetime.now() - start, file=file)

In [ ]:
with execution_time_printed():
    print("inside")
    import time
    time.sleep(0.5)
print("outside")

### Reentrant contextmanagers

In [ ]:
file = open("irrelevant.txt","w")
with file:
    file.write("a")

    
with file:
    file.write("a")

In [ ]:
import sqlite3
db = sqlite3.connect(":memory:")
with db:
    db.execute("")
    db.execute("")

In [ ]:
with db:
    db.execute("")
    db.execute("")

### Btw threading pool context manager

In [ ]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=1) as executor:
    future = executor.submit(pow, 323, 1235)

print(future.result())

In [ ]:
from concurrent.futures import ProcessPoolExecutor, Executor

with ProcessPoolExecutor(max_workers=8) as executor:
    executor: Executor
    powers = list(executor.map(pow, range(100), range(100)))
powers

### Multiple inheritance, method resolution order (mro)

In [ ]:
class A:
    def f(self):
        print("A")

class B:
    def f(self):
        print("B")

class C(B,A):
    pass

C().f()                             
print(C.mro())

In [ ]:
class C(A, B):
    pass

C().f()                             
print(C.mro())

In [50]:
class C(A, B):
    def f(self):
        B.f(self) # in general class.method(self) == object.method()

C().f()

NameError: name 'A' is not defined

### debugging - pdb

### Asyncio

In [ ]:
import asyncio
import time


async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


async def main():
    task1 = asyncio.create_task(say_after(1, "hello"))

    task2 = asyncio.create_task(say_after(2, "world"))

    print(f"started at {time.strftime('%X')}")

    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    await task1
    await task2

    print(f"finished at {time.strftime('%X')}")

await main()

Normally you would use ```asyncio.run(main())```, jupyter (IPython) is already running an event loop

In [ ]:
import time
from queue import Queue
from threading import Thread


def producer_func(queue):
    print(f"Putting {1}")
    queue.put(1)
    print(f"Putting {2}")
    queue.put(2)
    print(f"Producer waiting for more tasks")
    time.sleep(2)
    print(f"Putting {3}")
    queue.put(3)
    print(f"Producer shuting down")
    queue.put(None)


def consumer_func(queue):
    while True:
        task = queue.get()
        if task is None:
            print(f"Got None, exiting")
            queue.task_done()
            break
        time.sleep(0.5)
        queue.task_done()
        print(f"Task Done {task}")

queue = Queue()
producer = Thread(target=producer_func(queue))
consumer = Thread(target=consumer_func(queue))
producer.start()
consumer.start()
queue.join()

In [ ]:
import asyncio
import time


async def producer(queue):
    print(f"Putting {1}")
    await queue.put(1)
    print(f"Putting {2}")
    await queue.put(2)
    print(f"Producer waiting for more tasks")
    await asyncio.sleep(2)
    print(f"Putting {3}")
    await queue.put(3)
    print(f"Producer shuting down")
    await queue.put(None)


async def consumer(queue):
    while True:
        task = await queue.get()
        if task is None:
            print(f"Got None, exiting")
            queue.task_done()
            break
        time.sleep(0.5)
        queue.task_done()
        print(f"Task Done {task}")


async def main():
    queue = asyncio.queues.Queue()
    producer_coro = asyncio.create_task(producer(queue))
    consumer_coro = asyncio.create_task(consumer(queue))
    await producer_coro
    await consumer_coro
    await queue.join()


await main()


### Itertools

In [ ]:
import itertools

In [ ]:
list(itertools.chain([1, 2], (3, 4, 5), "6"))

In [ ]:
list(itertools.repeat(1,5))

In [ ]:
list(itertools.islice(itertools.count(), 4))

### Enumerate

In [ ]:
values = ["a","b"]

In [ ]:
# DONT DO THIS:
for i in range(len(values)):
    print(f"values[{i}] = {values[i]}")

In [ ]:
for index, value in enumerate(values):
    print(f"values[{index}] = {value}")

In [ ]:
list(enumerate(values)) == [(0, "a"), (1, "b")]

# TODO: dynamic class creation

# TODO: virtualenv, python -m

# TODO: Imports :(

# TODO: dynamic class creation

# TODO: microbenchmarking

### super() TODO

In [ ]:
class Square(Rectangle):
    def __init__(self, length):
        super().__init__(length, length)

class Cube(Square):
    def surface_area(self):
        face_area = super().area()
        return face_area * 6

    def volume(self):
        face_area = super().area()
        return face_area * self.length